In [19]:
import tkinter as tk
from tkinter import filedialog

def seleccionar_archivos():
    """
    Abre una ventana emergente para seleccionar uno o varios archivos.
    Retorna una lista con las rutas de los archivos seleccionados.
    """
    # 1. Inicializamos Tkinter y ocultamos la ventana raíz
    root = tk.Tk()
    root.withdraw()

    # 2. Aseguramos que la ventana aparezca frente a las demás
    root.attributes('-topmost', True)

    # 3. Abrimos el diálogo (askopenfilenames permite selección múltiple)
    # Puedes filtrar por extensiones específicas en 'filetypes'
    rutas = filedialog.askopenfilenames(
        title="Selecciona los archivos para el algoritmo",
        filetypes=(("Archivos de datos", "*.csv *.txt *.xlsx"), ("Todos los archivos", "*.*"))
    )

    # 4. Destruimos la instancia de root para liberar memoria
    root.destroy()

    # Convertimos la tupla resultante en una lista para facilitar su uso
    return list(rutas)

archivos = seleccionar_archivos()
print(len(archivos))

54


In [20]:
correcciones = [
    " - ",
    "- ",
    " -",
    "--",
    "  ",
    "   ",
    "R1 ",
    "R2 ",
    "Rack 1-",
    "Rack1-",
    "Rack2-",
    "Rack3-",
    "Rack4-",
    "Rack 2-",
    "iJF Small-",
    "IJF-",
    "==",
]
cambios = [
    ["Nev.", "Nevera"],
    ["ExhibidorCongelados", "Exhibidor Congelados"]
]

In [81]:
contenido = []
try:
    for ruta in archivos:
        with open(ruta, "r",encoding='utf-8') as file:
            # Eliminar dobles saltos de linea
            datos = file.read().replace("\n\n","\n")
            # Separo contenido por cada salto de linea
            datos = datos.split("\n")
            # Extraigo los datos de nombre y equipos de la linea 2 y 4
            datos = datos[2:5:2]
            # Elimino ; del nombre del local
            datos[0] = datos[0].replace(";","")
            # Separo los nombres de los equipos con el caranter ';' y elimino ultimo espacio en blanco 
            for j in correcciones:
                if j in correcciones[0:4]:
                    datos[1] = datos[1].replace(j, "-")
                else:
                    datos[1] = datos[1].replace(j, "")
            
            for x in cambios:
                datos[1] = datos[1].replace(x[0], x[1])
            
            datos[1] = datos[1].split(";")
            datos[1].pop()
            
            # Modificar algo de cada nombre del equipo
            # for y in datos[1]:
            
            # Guardo los datos procesados
            contenido.append(datos)
        
except Exception as E:
    print(E)

In [82]:
contenido
len(contenido)

5

In [1]:
import sys
import os

# Obtener la ruta absoluta de la carpeta que contiene el paquete
# Ajusta 'ruta/a/tu/carpeta_externa' a tu estructura real
ruta_paquete = os.path.abspath('D:\Proyecto Automatizar Informe\creacion de base datos')

# Añadir la ruta al sys.path
if ruta_paquete not in sys.path:
    sys.path.append(ruta_paquete)
    
# Ahora puedes importar como si estuviera en la misma carpeta
from src.Obtener_Datos import Abrir_archivos, Data

In [2]:
archivos = Abrir_archivos.seleccionar_archivos()
print(len(archivos))

54


In [3]:
a = Data.procesar_Data(archivos)
print(a)

['REY 4 ALTOS', ['MT1-CAMARA CARNES', 'MT2-CAMARA LACTEOS', 'MT3-CAMARA LEGUMBRES', 'MT4A-EXHIBIDOR LACTEOS', 'MT4B-EXHIBIDOR LACTEOS', 'MT4C-EXHIBIDOR LACTEOS', 'MT5.1A-EXHIBIDOR LACTEOS', 'MT5.1B-EXHIBIDOR LACTEOS', 'MT5.2A-EXHIBIDOR LACTEOS', 'MT5.2B-EXHIBIDOR LACTEOS', 'MT6-EXHIBIDOR PUNTERA POLLOS', 'MT7.1A-EXHIBIDOR CARNES', 'MT7.1B-EXHIBIDOR CARNES', 'MT7.2A-EXHIBIDOR CARNES', 'MT7.2B-EXHIBIDOR CARNES', 'MT8-EXHIBIDOR LEGUMBRES', 'MT9.1A-EXHIBIDOR LEGUMBRES', 'MT9.1B-EXHIBIDOR LEGUMBRES', 'MT9.2A-EXHIBIDOR LEGUMBRES', 'MT9.2B-EXHIBIDOR LEGUMBRES', 'MT10-EXHIBIDOR TORTAS', 'MT11.1A-EXHIBIDOR CARNES', 'MT11.1B-EXHIBIDOR CARNES', 'MT12.1A-EXHIBIDOR BEBIDAS', 'MT12.1B-EXHIBIDOR BEBIDAS', 'MT12.2A-EXHIBIDOR BEBIDAS', 'MT12.2B-EXHIBIDOR BEBIDAS', 'MT13.1A-EXHIBIDOR BEBIDAS', 'MT13.1B-EXHIBIDOR BEBIDAS', 'MT13.2A-EXHIBIDOR BEBIDAS', 'MT13.2B-EXHIBIDOR BEBIDAS', 'MT14-EXHIBIDOR EMBUTIDOS DESHABILITADO', 'MT15A-EXHIBIDOR DELI', 'MT15B-EXHIBIDOR DELI', 'MT15C-EXHIBIDOR DELI', 'MT16.1-EXHI

In [4]:
# Ingresar a los nombres
def cambio_nombres(lista):
    for local in lista:
        for equipos in local[1]:
            print(equipos)

cambio_nombres(a[0:2])

MT1-CAMARA CARNES
MT2-CAMARA LACTEOS
MT3-CAMARA LEGUMBRES
MT4A-EXHIBIDOR LACTEOS
MT4B-EXHIBIDOR LACTEOS
MT4C-EXHIBIDOR LACTEOS
MT5.1A-EXHIBIDOR LACTEOS
MT5.1B-EXHIBIDOR LACTEOS
MT5.2A-EXHIBIDOR LACTEOS
MT5.2B-EXHIBIDOR LACTEOS
MT6-EXHIBIDOR PUNTERA POLLOS
MT7.1A-EXHIBIDOR CARNES
MT7.1B-EXHIBIDOR CARNES
MT7.2A-EXHIBIDOR CARNES
MT7.2B-EXHIBIDOR CARNES
MT8-EXHIBIDOR LEGUMBRES
MT9.1A-EXHIBIDOR LEGUMBRES
MT9.1B-EXHIBIDOR LEGUMBRES
MT9.2A-EXHIBIDOR LEGUMBRES
MT9.2B-EXHIBIDOR LEGUMBRES
MT10-EXHIBIDOR TORTAS
MT11.1A-EXHIBIDOR CARNES
MT11.1B-EXHIBIDOR CARNES
MT12.1A-EXHIBIDOR BEBIDAS
MT12.1B-EXHIBIDOR BEBIDAS
MT12.2A-EXHIBIDOR BEBIDAS
MT12.2B-EXHIBIDOR BEBIDAS
MT13.1A-EXHIBIDOR BEBIDAS
MT13.1B-EXHIBIDOR BEBIDAS
MT13.2A-EXHIBIDOR BEBIDAS
MT13.2B-EXHIBIDOR BEBIDAS
MT14-EXHIBIDOR EMBUTIDOS DESHABILITADO
MT15A-EXHIBIDOR DELI
MT15B-EXHIBIDOR DELI
MT15C-EXHIBIDOR DELI
MT16.1-EXHIBIDOR CARNES ASISTIDO
MT16.2-EXHIBIDOR CARNES AUTO
MT16.3-EXHIBIDOR CARNES ASISTIDO
MT16.4-EXHIBIDOR CARNES AUTO
MT17.1-EXH